In [ ]:
import os
import pathlib
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import dash_daq as daq
import pandas as pd
import base64
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import random
import joblib
import cv2
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
from sklearn.metrics import roc_curve, auc,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import dash_table

app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
app.title = "FOUR SIGMA"
server = app.server
app.config["suppress_callback_exceptions"] = True

df = pd.read_csv("C:/Users/admin/Desktop/Final/dash_5/data/final.csv")

DecisionTree = joblib.load("model_DT.pkl")
LogisticRegression = joblib.load("model_lr.pkl")
RandomForest = joblib.load("model_rg.pkl")
XGboost = joblib.load("my_model.pkl")

def create_df(x):
    df = pd.read_csv('C:/dash_7/data/final.csv')
    res = df[0+1518*x:1518*(x+1)]
    return res

df = create_df(0)

model_1 = keras.models.load_model("cnn_ok_def.h5") 
model_2 = keras.models.load_model("cnn_def123.h5")
test_cases_dent_side =os.listdir(os.path.join(os.getcwd(), 'Split_data/test/Dent_Side_Augmentation'))
test_cases_dent = os.listdir(os.path.join(os.getcwd(), "Split_data/test/Dent_Augmentation"))
test_cases_side = os.listdir(os.path.join(os.getcwd(), "Split_data/test/Side_Augmentation"))
test_cases_ok = os.listdir(os.path.join(os.getcwd(), "Split_data/test/OK_data"))
total_test_case = test_cases_ok + test_cases_side + test_cases_dent + test_cases_dent_side 
#random.shuffle(total_test_case)
image_shape = (300,300,1)
plt.figure(figsize=(8,8))

cnt = [0]
cntt = [0]
cnt_1 = [0]
cnt_2 = [0]
cnt_3 = [0]


def convert(res):
    ypred = []
    result = {}
    plt.savefig(f'assets/{res}')
    if res in test_cases_dent_side:
        path = 'Dent_Side_Augmentation/'
        result['actual'] = '불량'

    elif res in test_cases_dent:
        path = 'Dent_Augmentation/'
        result['actual'] = '불량'

    elif res in test_cases_side:
        path = 'Side_Augmentation/'
        result['actual'] = '불량'

    else:
        path = 'OK_data/'
        result['actual'] = '정상'        

    img_pred = cv2.imread("C:/dash_7/Split_data/test/"+path+res, cv2.IMREAD_GRAYSCALE)
    img_pred = cv2.resize(img_pred, (300, 300))
    img_pred = img_pred.reshape(1, *image_shape)
    img_pred = img_pred / 255 # rescale
    prediction_1 = model_1.predict(img_pred)
    prediction_2 = model_2.predict(img_pred)    
    
    img = cv2.imread("C:/dash_7/Split_data/test/"+path+res)
    label = res.split("_")[0]
    index=np.argmax(prediction_2)
    ypred.append(index)    
    if result['actual'] == '불량':
        if (ypred[0]==0):
            result['predicted_label'] = "내부결함"
            prob = (1-prediction_2[0,0]) * 100
            cnt[0] +=1
            cnt_1[0] +=1
        elif (ypred[0]==2):
            result['predicted_label'] = "내부 + 외부결함"
            prob = (1-prediction_2[0,2]) * 100
            cnt[0] +=1
            cnt_2[0] +=1
        elif (ypred[0]==1):
            result['predicted_label'] = "외부결함"
            prob = (1-prediction_2[0,1]) * 100
            cnt[0] +=1
            cnt_3[0] +=1
    else :
        result['predicted_label'] ="PASS"
        cntt[0] +=1      
           
    plt.imshow(img,cmap='gray')
    plt.axis('off')
    plt.savefig(f'assets/{res}')
    result['res'] = res
    result['count1'] = cnt
    result['countt1'] = cntt
    result['count_1'] = cnt_1
    result['count_2'] = cnt_2
    result['count_3'] = cnt_3  
    return result

params = ['Batch', 'pH', 'Temp', 'Voltage', 'Eduction']
max_length = len(df)

suffix_row = "_row"
suffix_button_id = "_button"
suffix_sparkline_graph = "_sparkline_graph"
suffix_count = "_count"
suffix_ooc_n = "_OOC_number"
suffix_ooc_g = "_OOC_graph"
suffix_indicator = "_indicator"

#이미지 
model = keras.models.load_model("my_h5_model.h5") 
test_cases_dent_side =os.listdir(os.path.join(os.getcwd(), 'Split_data/test/Dent_Side_Augmentation'))
test_cases_dent = os.listdir(os.path.join(os.getcwd(), "Split_data/test/Dent_Augmentation"))
test_cases_side = os.listdir(os.path.join(os.getcwd(), "Split_data/test/Side_Augmentation"))
test_cases_ok = os.listdir(os.path.join(os.getcwd(), "Split_data/test/OK_data"))
total_test_case = test_cases_ok + test_cases_side + test_cases_dent + test_cases_dent_side 
random.shuffle(total_test_case)
image_shape = (300,300,1)
plt.figure(figsize=(8,8))
res = random.choice(total_test_case)
#res = total_test_case

cnt = [0,0]
cnt2 = [0,0,0]




def build_banner():
    return html.Div(
        id="banner",
        className="banner",
        children=[
            html.Div(
                id="banner-text",
                children=[
                    html.H5("크롬도금 품질분류 및 공정효율화 대시보드"),
                    html.H6("Chromate process & Quality control Dashboard"),
                ],
            ),
            html.Div(
                id="banner-logo",
                children=[
                    html.A(
                        html.Button(children="FOUR SIGMA"),
                        href="https://plotly.com/get-demo/",
                    ),
                    html.Button(
                        id="learn-more-button", children="LEARN MORE", n_clicks=0
                    ),
                    html.A(
                        html.Img(id="logo", src=app.get_asset_url("dash-logo-new.png")),
                        href="https://plotly.com/dash/",
                    ),
                ],
            ),
        ],
    )


def build_tabs():
    return html.Div(
        id="tabs",
        className="tabs",
        children=[
            dcc.Tabs(
                id="app-tabs",
                value="tab1",
                className="custom-tabs",
                children=[
                    dcc.Tab(
                        id="Live-chart-tab",
                        label="Live Chart Dashboard",
                        value="tab0",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id="Specs-tab",
                        label="Specification Settings",
                        value="tab1",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id="Control-chart-tab",
                        label="Control Charts Dashboard",
                        value="tab2",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                    ),
                    dcc.Tab(
                        id="data-analysis-tab",
                        label="analysis Dashboard",
                        value="tab3",
                        className="custom-tab",
                        selected_className="custom-tab--selected",
                    ),
                ],
            )
        ],
    )


def init_df():
    ret = {}
    for col in list(df[1:]):
        data = df[col]
        stats = data.describe()

        std = stats["std"].tolist()
        ucl = (stats["mean"] + 3 * stats["std"]).tolist()
        lcl = (stats["mean"] - 3 * stats["std"]).tolist()
        usl = (stats["mean"] + stats["std"]).tolist()
        lsl = (stats["mean"] - stats["std"]).tolist()

        ret.update(
            {
                col: {
                    "count": stats["count"].tolist(),
                    "data": data,
                    "mean": stats["mean"].tolist(),
                    "std": std,
                    "ucl": round(ucl, 3),
                    "lcl": round(lcl, 3),
                    "usl": round(usl, 3),
                    "lsl": round(lsl, 3),
                    "min": stats["min"].tolist(),
                    "max": stats["max"].tolist(),
                    "ooc": populate_ooc(data, ucl, lcl),
                }
            }
        )

    return ret


def populate_ooc(data, ucl, lcl):
    ooc_count = 0
    ret = []
    for i in range(len(data)):
        if data[i] >= ucl or data[i] <= lcl:
            ooc_count += 1
            ret.append(ooc_count / (i + 1))
        else:
            ret.append(ooc_count / (i + 1))
    return ret


state_dict = init_df()


def init_value_setter_store():
    # Initialize store data
    state_dict = init_df()
    return state_dict


def build_tab_1():
    return [
        # Manually select metrics
        html.Div(
            id="set-specs-intro-container",
            # className='twelve columns',
            children=[html.P("관리한계와 규격한계를 설정하여 공정 이상탐지가 가능하도록 도와줍니다."),
            html.H1("this page helps to detect process anomalies by setting control limits and specification limits")]
        ),
        html.Div(
            id="settings-menu",
            children=[
                html.Div(
                    id="metric-select-menu",
                    # className='five columns',
                    children=[
                        html.Label(id="metric-select-title", children="변수 설정"),
                        html.Br(),
                        dcc.Dropdown(
                            id="metric-select-dropdown",
                            options=list(
                                {"label": param, "value": param} for param in params[1:]
                            ),
                            value=params[1],
                        ),
                    ],
                ),
                html.Div(
                    id="value-setter-menu",
                    # className='six columns',
                    children=[
                        html.Div(id="value-setter-panel"),
                        html.Br(),
                        html.Div(
                            id="button-div",
                            children=[
                                html.Button("Update", id="value-setter-set-btn"),
                                html.Button(
                                    "View current setup",
                                    id="value-setter-view-btn",
                                    n_clicks=0,
                                ),
                            ],
                        ),
                    ],
                ),
            ],
        ),
    ]


ud_usl_input = daq.NumericInput(
    id="ud_usl_input", className="setting-input", size=200, max=9999999
)
ud_lsl_input = daq.NumericInput(
    id="ud_lsl_input", className="setting-input", size=200, max=9999999
)
ud_ucl_input = daq.NumericInput(
    id="ud_ucl_input", className="setting-input", size=200, max=9999999
)
ud_lcl_input = daq.NumericInput(
    id="ud_lcl_input", className="setting-input", size=200, max=9999999
)

def build_tab_2():
    return [
        html.Div(
        id="third-section-1", className="row",
        children=[
            html.Div(
                id="image-1", className="column",
                children=[

                    html.P("image detect"),
                    html.Div(
                        id="button-div2",
                        children=[
                            html.Img(id= "image", src='',style={"width": "350px", "height": "350px"}),
                            html.Div(
                                id="button-div3",
                                children=[
                                    daq.StopButton(id="stop-button", size=150, n_clicks=0)],)
                        ],
                    ),
        ],
            ),
            html.Div(
                id ="image-2", className="column",
                children=[
                    html.P("Text detect",),
                    html.H1(id='result1'),
                    html.H1(id='result2'),
                ],
            ),
            html.Div(
                 id ="image-3",
                className="column",
                children=[dcc.Tabs(className="third-tabs",
                                   children=[
                                       dcc.Tab(
                                           id="Live-chart-tab",
                                           label='불량개수 체크',
                                           className="custom-tab",
                                           selected_className="custom-tab--selected",
                                           children=[generate_imagechart(),
                                                    ]),
                                       dcc.Tab(
                                           id="Live-chart-tab",
                                           label='불량패턴개수 체크',
                                           className="custom-tab",
                                           selected_className="custom-tab--selected",
                                           children=[generate_imagechart2(),
                                                    ]),
                
                ]),])          
      ,],),
]
@app.callback(
    output=[Output("image", "src"),
            Output("result1","children"),
            Output("result2","children"),
            Output("imagegraph_1","figure"),
            Output("imagegraph_2","figure"),],
    inputs=[Input("interval-component", "n_intervals")],)
def input_image(interval):  
    if interval >= max_length:
        total_count = max_length - 1
    else:
        total_count = interval - 1
    res = total_test_case[total_count]
    output = convert(res)
    result = app.get_asset_url(output['res'])
    values1 = output['count1']
    vakues11 = output['countt1']
    values_1 = output['count_1']
    values_2 = output['count_2']
    values_3 = output['count_3']    
    new_figure1 = {
                    'data':[
                        {'x':[2],'y':values1,
                        'type':'bar','name':'불량'},
                        {'x':[1],'y':vakues11,
                        'type':'bar','name':'정상'},
                        

                        
                    ],
    "layout": {
                        "paper_bgcolor":"rgba(30,33,48,0)",
                        "plot_bgcolor":"rgba(30,33,48,0)",
        "font":{"color":"white"},
                    }}
    new_figure2 = {'data':[{'x':[1],'y':values_1,'type':'bar','name':'내부'},
                           {'x':[2],'y':values_2,'type':'bar','name':'내부+외부'},
                           {'x':[3],'y':values_3,'type':'bar','name':'외부'},
                          ],
                      "layout": {
                        "paper_bgcolor":"rgba(30,33,48,0)",
                        "plot_bgcolor":"rgba(30,33,48,0)",
        "font":{"color":"white"},
                    }}
    return result,output['actual'],output['predicted_label'],new_figure1, new_figure2


def build_value_setter_line(line_num, label, value, col3):
    return html.Div(
        id=line_num,
        children=[
            html.Label(label, className="four columns"),
            html.Label(value, className="four columns"),
            html.Div(col3, className="four columns"),
        ],
        className="row",
    )


def generate_modal():
    return html.Div(
        id="markdown",
        className="modal",
        children=(
            html.Div(
                id="markdown-container",
                className="markdown-container",
                children=[
                    html.Div(
                        className="close-container",
                        children=html.Button(
                            "Close",
                            id="markdown_close",
                            n_clicks=0,
                            className="closeButton",
                        ),
                    ),
                    html.Div(
                        className="markdown-text",
                        children=dcc.Markdown(
                            children=(
                                """
                        ###### What is this mock app about?
                        This is a dashboard for monitoring real-time process quality along manufacture production line.
                        ###### What does this app shows
                        Click on buttons in `Parameter` column to visualize details of measurement trendlines on the bottom panel.
                        The sparkline on top panel and control chart on bottom panel show Shewhart process monitor using mock data.
                        The trend is updated every other second to simulate real-time measurements. Data falling outside of six-sigma control limit are signals indicating 'Out of Control(OOC)', and will
                        trigger alerts instantly for a detailed checkup.
                        
                        Operators may stop measurement by clicking on `Stop` button, and edit specification parameters by clicking specification tab.
                        ###### Source Code
                        You can find the source code of this app on our [Github repository](https://github.com/plotly/dash-sample-apps/tree/main/apps/dash-manufacture-spc-dashboard).
                    """
                            )
                        ),
                    ),
                ],
            )
        ),
    )


def build_quick_stats_panel():
    return html.Div(
        id="quick-stats",
        className="row",
        children=[
            html.Div(
                id="card-1",
                children=[
                    html.P("Operator ID"),
                    daq.LEDDisplay(
                        id="operator-led",
                        value="1704",
                        color="#92e0d3",
                        backgroundColor="#1e2130",
                        size=50,
                    ),
                ],
            ),
            html.Div(
                id="card-2",
                children=[
                    html.P("Time to completion"),
                    daq.Gauge(
                        id="progress-gauge",
                        max=200,
                        min=0,
                        units="개수",
                        showCurrentValue=True,  # default size 200 pixel
                    ),
                ],
            ),
            html.Div(
                id="utility-card",
                children=[daq.StopButton(id="stop-button", size=160, n_clicks=0)],
            ),
        ],
    )


def generate_section_banner(title):
    return html.Div(className="section-banner", children=title)


def build_top_panel(stopped_interval):
    return html.Div(
        id="top-section-container",
        className="row",
        children=[
            # Metrics summary
            html.Div(
                id="metric-summary-session",
                className="eight columns",
                children=[
                    generate_section_banner("Process Control Metrics Summary"),
                    html.Div(
                        id="metric-div",
                        children=[
                            generate_metric_list_header(),
                            html.Div(
                                id="metric-rows",
                                children=[
                                    generate_metric_row_helper(stopped_interval, 1),
                                    generate_metric_row_helper(stopped_interval, 2),
                                    generate_metric_row_helper(stopped_interval, 3),
                                    generate_metric_row_helper(stopped_interval, 4),
                                ],
                            ),
                        ],
                    ),
                ],
            ),
            # Piechart
            html.Div(
                id="ooc-piechart-outer",
                className="four columns",
                children=[
                    generate_section_banner("% OOC per Parameter"),
                    generate_piechart(),
                ],
            ),
        ],
    )

def build_table_graph_1(stopped_interval):
    return html.Div(
        id="top-section-container",
        className="row",
        children=[
            # Metrics summary
            html.Div(
                id="graph_1",
                className="eight columns",
                children=[
                    generate_section_banner("정형 데이터 효율 그래프"),
                    generate_linechart(),
                ],
            ),
            # Table
            html.Div(
                id="table_1",
                className="four columns",
                children=[
                    generate_section_banner("정형 데이터 알람 테이블"),
                    generate_table(),
                ],
            ),
        ],
    )

def generate_piechart():
    return dcc.Graph(
        id="piechart",
        figure={
            "data": [
                {
                    "labels": [],
                    "values": [],
                    "type": "pie",
                    "marker": {"line": {"color": "white", "width": 1}},
                    "hoverinfo": "label",
                    "textinfo": "label",
                }
            ],
            "layout": {
                "margin": dict(l=20, r=20, t=20, b=20),
                "showlegend": True,
                "paper_bgcolor": "rgba(0,0,0,0)",
                "plot_bgcolor": "rgba(0,0,0,0)",
                "font": {"color": "white"},
                "autosize": True,
            },
        },
    )


# Build header
def generate_metric_list_header():
    return generate_metric_row(
        "metric_header",
        {"height": "3rem", "margin": "1rem 0", "textAlign": "center"},
        {"id": "m_header_1", "children": html.Div("Parameter")},
        {"id": "m_header_2", "children": html.Div("Count")},
        {"id": "m_header_3", "children": html.Div("Sparkline")},
        {"id": "m_header_4", "children": html.Div("OOC%")},
        {"id": "m_header_5", "children": html.Div("%OOC")},
        {"id": "m_header_6", "children": "Pass/Fail"},
    )


def generate_metric_row_helper(stopped_interval, index):
    item = params[index]

    div_id = item + suffix_row
    button_id = item + suffix_button_id
    sparkline_graph_id = item + suffix_sparkline_graph
    count_id = item + suffix_count
    ooc_percentage_id = item + suffix_ooc_n
    ooc_graph_id = item + suffix_ooc_g
    indicator_id = item + suffix_indicator

    return generate_metric_row(
        div_id,
        None,
        {
            "id": item,
            "className": "metric-row-button-text",
            "children": html.Button(
                id=button_id,
                className="metric-row-button",
                children=item,
                title="Click to visualize live SPC chart",
                n_clicks=0,
            ),
        },
        {"id": count_id, "children": "0"},
        {
            "id": item + "_sparkline",
            "children": dcc.Graph(
                id=sparkline_graph_id,
                style={"width": "100%", "height": "95%"},
                config={
                    "staticPlot": False,
                    "editable": False,
                    "displayModeBar": False,
                },
                figure=go.Figure(
                    {
                        "data": [
                            {
                                "x": state_dict["Batch"]["data"].tolist()[
                                    :stopped_interval
                                ],
                                "y": state_dict[item]["data"][:stopped_interval],
                                "mode": "lines+markers",
                                "name": item,
                                "line": {"color": "#f4d44d"},
                            }
                        ],
                        "layout": {
                            "uirevision": True,
                            "margin": dict(l=0, r=0, t=4, b=4, pad=0),
                            "xaxis": dict(
                                showline=False,
                                showgrid=False,
                                zeroline=False,
                                showticklabels=False,
                            ),
                            "yaxis": dict(
                                showline=False,
                                showgrid=False,
                                zeroline=False,
                                showticklabels=False,
                            ),
                            "paper_bgcolor": "rgba(0,0,0,0)",
                            "plot_bgcolor": "rgba(0,0,0,0)",
                        },
                    }
                ),
            ),
        },
        {"id": ooc_percentage_id, "children": "0.00%"},
        {
            "id": ooc_graph_id + "_container",
            "children": daq.GraduatedBar(
                id=ooc_graph_id,
                color={
                    "ranges": {
                        "#92e0d3": [0, 2],
                        "#f4d44d ": [2, 4],
                        "#f45060": [4, 15],
                    }
                },
                showCurrentValue=False,
                max=15,
                value=0,
            ),
        },
        {
            "id": item + "_pf",
            "children": daq.Indicator(
                id=indicator_id, value=True, color="#91dfd2", size=12
            ),
        },
    )


def generate_metric_row(id, style, col1, col2, col3, col4, col5, col6):
    if style is None:
        style = {"height": "8rem", "width": "100%"}

    return html.Div(
        id=id,
        className="row metric-row",
        style=style,
        children=[
            html.Div(
                id=col1["id"],
                className="one column",
                style={"margin-right": "2.5rem", "minWidth": "50px"},
                children=col1["children"],
            ),
            html.Div(
                id=col2["id"],
                style={"textAlign": "center"},
                className="one column",
                children=col2["children"],
            ),
            html.Div(
                id=col3["id"],
                style={"height": "100%"},
                className="four columns",
                children=col3["children"],
            ),
            html.Div(
                id=col4["id"],
                style={},
                className="one column",
                children=col4["children"],
            ),
            html.Div(
                id=col5["id"],
                style={"height": "100%", "margin-top": "5rem"},
                className="three columns",
                children=col5["children"],
            ),
            html.Div(
                id=col6["id"],
                style={"display": "flex", "justifyContent": "center"},
                className="one column",
                children=col6["children"],
            ),
        ],
    )


def build_chart_panel():
    return html.Div(
        id="control-chart-container",
        className="twelve columns",
        children=[
            generate_section_banner("Live SPC Chart"),
            dcc.Graph(
                id="control-chart-live",
                figure=go.Figure(
                    {
                        "data": [
                            {
                                "x": [],
                                "y": [],
                                "mode": "lines+markers",
                                "name": params[1],
                            }
                        ],
                        "layout": {
                            "paper_bgcolor": "rgba(0,0,0,0)",
                            "plot_bgcolor": "rgba(0,0,0,0)",
                            "xaxis": dict(
                                showline=False, showgrid=False, zeroline=False
                            ),
                            "yaxis": dict(
                                showgrid=False, showline=False, zeroline=False
                            ),
                            "autosize": True,
                        },
                    }
                ),
            ),
        ],
    )


def generate_graph(interval, specs_dict, col):
    stats = state_dict[col]
    col_data = stats["data"]
    mean = stats["mean"]
    ucl = specs_dict[col]["ucl"]
    lcl = specs_dict[col]["lcl"]
    usl = specs_dict[col]["usl"]
    lsl = specs_dict[col]["lsl"]

    x_array = state_dict["Batch"]["data"].tolist()
    y_array = col_data.tolist()

    total_count = 0

    if interval > max_length:
        total_count = max_length - 1
    elif interval > 0:
        total_count = interval

    ooc_trace = {
        "x": [],
        "y": [],
        "name": "Out of Control",
        "mode": "markers",
        "marker": dict(color="rgba(210, 77, 87, 0.7)", symbol="square", size=11),
    }

    for index, data in enumerate(y_array[:total_count]):
        if data >= ucl or data <= lcl:
            ooc_trace["x"].append(index + 1)
            ooc_trace["y"].append(data)

    histo_trace = {
        "x": x_array[:total_count],
        "y": y_array[:total_count],
        "type": "histogram",
        "orientation": "h",
        "name": "Distribution",
        "xaxis": "x2",
        "yaxis": "y2",
        "marker": {"color": "#f4d44d",'line':{'color':'#161a28', 'width':3}},
    }

    fig = {
        "data": [
            {
                "x": x_array[:total_count],
                "y": y_array[:total_count],
                "mode": "lines+markers",
                "name": col,
                "line": {"color": "#f4d44d"},
            },
            ooc_trace,
            histo_trace,
        ]
    }

    len_figure = len(fig["data"][0]["x"])

    fig["layout"] = dict(
        margin=dict(t=40),
        hovermode="closest",
        uirevision=col,
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        legend={"font": {"color": "darkgray"}, "orientation": "h", "x": 0, "y": 1.1},
        font={"color": "darkgray"},
        showlegend=True,
        xaxis={
            "zeroline": False,
            "showgrid": False,
            "title": "Batch Number",
            "showline": False,
            "domain": [0, 0.8],
            "titlefont": {"color": "darkgray"},
        },
        yaxis={
            "title": col,
            "showgrid": False,
            "showline": False,
            "zeroline": False,
            "autorange": True,
            "titlefont": {"color": "darkgray"},
        },
        annotations=[
            {
                "x": 0.75,
                "y": lcl,
                "xref": "paper",
                "yref": "y",
                "text": "LCL:" + str(round(lcl, 3)),
                "showarrow": False,
                "font": {"color": "white"},
            },
            {
                "x": 0.75,
                "y": ucl,
                "xref": "paper",
                "yref": "y",
                "text": "UCL: " + str(round(ucl, 3)),
                "showarrow": False,
                "font": {"color": "white"},
            },
            {
                "x": 0.75,
                "y": usl,
                "xref": "paper",
                "yref": "y",
                "text": "USL: " + str(round(usl, 3)),
                "showarrow": False,
                "font": {"color": "white"},
            },
            {
                "x": 0.75,
                "y": lsl,
                "xref": "paper",
                "yref": "y",
                "text": "LSL: " + str(round(lsl, 3)),
                "showarrow": False,
                "font": {"color": "white"},
            },
            {
                "x": 0.75,
                "y": mean,
                "xref": "paper",
                "yref": "y",
                "text": "Targeted mean: " + str(round(mean, 3)),
                "showarrow": False,
                "font": {"color": "white"},
            },
        ],
        shapes=[
            {
                "type": "line",
                "xref": "x",
                "yref": "y",
                "x0": 1,
                "y0": usl,
                "x1": len_figure + 1,
                "y1": usl,
                "line": {"color": "#91dfd2", "width": 1, "dash": "dot"},
            },
            {
                "type": "line",
                "xref": "x",
                "yref": "y",
                "x0": 1,
                "y0": lsl,
                "x1": len_figure + 1,
                "y1": lsl,
                "line": {"color": "#91dfd2", "width": 1, "dash": "dot"},
            },
            {
                "type": "line",
                "xref": "x",
                "yref": "y",
                "x0": 1,
                "y0": ucl,
                "x1": len_figure + 1,
                "y1": ucl,
                "line": {"color": "rgb(255,127,80)", "width": 1, "dash": "dot"},
            },
            {
                "type": "line",
                "xref": "x",
                "yref": "y",
                "x0": 1,
                "y0": mean,
                "x1": len_figure + 1,
                "y1": mean,
                "line": {"color": "rgb(255,127,80)", "width": 2},
            },
            {
                "type": "line",
                "xref": "x",
                "yref": "y",
                "x0": 1,
                "y0": lcl,
                "x1": len_figure + 1,
                "y1": lcl,
                "line": {"color": "rgb(255,127,80)", "width": 1, "dash": "dot"},
            },
        ],
        xaxis2={
            "title": "Count",
            "domain": [0.8, 1],  # 70 to 100 % of width
            "titlefont": {"color": "darkgray"},
            "showgrid": False,
        },
        yaxis2={
            "anchor": "free",
            "overlaying": "y",
            "side": "right",
            "showticklabels": False,
            "titlefont": {"color": "darkgray"},
        },
    )

    return fig


def update_sparkline(interval, param):
    x_array = state_dict["Batch"]["data"].tolist()
    y_array = state_dict[param]["data"].tolist()

    if interval == 0:
        x_new = y_new = None

    else:
        if interval >= max_length:
            total_count = max_length
        else:
            total_count = interval
        x_new = x_array[:total_count][-1]
        y_new = y_array[:total_count][-1]

    return dict(x=[[x_new]], y=[[y_new]]), [0]


def update_count(interval, col, data):
    if interval == 0:
        return "0", "0.00%", 0.00001, "#92e0d3"

    if interval > 0:

        if interval >= max_length:
            total_count = max_length - 1
        else:
            total_count = interval - 1

        ooc_percentage_f = data[col]["ooc"][total_count] * 100
        ooc_percentage_str = "%.2f" % ooc_percentage_f + "%"

        # Set maximum ooc to 15 for better grad bar display
        if ooc_percentage_f > 15:
            ooc_percentage_f = 15

        if ooc_percentage_f == 0.0:
            ooc_grad_val = 0.00001
        else:
            ooc_grad_val = float(ooc_percentage_f)

        # Set indicator theme according to threshold 5%
        if 0 <= ooc_grad_val <= 5:
            color = "#92e0d3"
        elif 5 < ooc_grad_val < 7:
            color = "#f4d44d"
        else:
            color = "#FF0000"

    return str(total_count + 1), ooc_percentage_str, ooc_grad_val, color


app.layout = html.Div(
    id="big-app-container",
    children=[
        build_banner(),
        dcc.Interval(
            id="interval-component",
            interval=2 * 1000,  # in milliseconds
            n_intervals=50,  # start at batch 50
            disabled=True,
        ),
        html.Div(
            id="app-container",
            children=[
                build_tabs(),
                # Main app
                html.Div(id="app-content"),
            ],
        ),
        dcc.Store(id="value-setter-store", data=init_value_setter_store()),
        #dcc.Store(id="n-interval-stage", data=1324),
        generate_modal(),
    ],
)


@app.callback(
    [Output("app-content", "children"), Output("interval-component", "n_intervals")],
    [Input("app-tabs", "value")],
    [State("interval-component", "n_intervals")],
)
def render_tab_content(tab_switch, stopped_interval):
    if tab_switch == "tab1":
        return build_tab_1(), stopped_interval
    if tab_switch == "tab3":
        return build_tab_2(), stopped_interval
    if tab_switch == "tab0":
        return (
        html.Div(
            id="status-container",
            children=[
                build_quick_stats_panel(), #오퍼, 시간 
                html.Div(
                    id="graphs-container",
                    children=[build_table_graph_1(stopped_interval)],
                ),
            ],
        ),
        stopped_interval,
    )
    return (
        html.Div(
            id="status-container",
            children=[
                build_quick_stats_panel(),
                html.Div(
                    id="graphs-container",
                    children=[build_top_panel(stopped_interval), build_chart_panel()],
                ),
            ],
        ),
        stopped_interval,
    )


# Update interval
@app.callback(
    Output("n-interval-stage", "data"),
    [Input("app-tabs", "value")],
    [
        State("interval-component", "n_intervals"),
        State("interval-component", "disabled"),
        State("n-interval-stage", "data"),
    ],
)
def update_interval_state(tab_switch, cur_interval, disabled, cur_stage):
    if disabled:
        return cur_interval

    if tab_switch == "tab1":
        return cur_interval
    return cur_stage


# Callbacks for stopping interval update
@app.callback(
    [Output("interval-component", "disabled"), Output("stop-button", "buttonText")],
    [Input("stop-button", "n_clicks")],
    [State("interval-component", "disabled")],
)
def stop_production(n_clicks, current):
    if n_clicks == 0:
        return True, "start"
    return not current, "stop" if current else "start"


# ======= Callbacks for modal popup =======
@app.callback(
    Output("markdown", "style"),
    [Input("learn-more-button", "n_clicks"), Input("markdown_close", "n_clicks")],
)
def update_click_output(button_click, close_click):
    ctx = dash.callback_context

    if ctx.triggered:
        prop_id = ctx.triggered[0]["prop_id"].split(".")[0]
        if prop_id == "learn-more-button":
            return {"display": "block"}

    return {"display": "none"}


# ======= update progress gauge =========
@app.callback(
    output=Output("progress-gauge", "value"),
    inputs=[Input("interval-component", "n_intervals")],
)
def update_gauge(interval):
    if interval < max_length:
        total_count = interval
    else:
        total_count = max_length

    return int(total_count)


# ===== Callbacks to update values based on store data and dropdown selection =====
@app.callback(
    output=[
        Output("value-setter-panel", "children"),
        Output("ud_usl_input", "value"),
        Output("ud_lsl_input", "value"),
        Output("ud_ucl_input", "value"),
        Output("ud_lcl_input", "value"),
    ],
    inputs=[Input("metric-select-dropdown", "value")],
    state=[State("value-setter-store", "data")],
)
def build_value_setter_panel(dd_select, state_value):
    return (
        [
            build_value_setter_line(
                "value-setter-panel-header",
                "Specs",
                "Historical Value",
                "Set new value",
            ),
            build_value_setter_line(
                "value-setter-panel-usl",
                "Upper Specification limit",
                state_dict[dd_select]["usl"],
                ud_usl_input,
            ),
            build_value_setter_line(
                "value-setter-panel-lsl",
                "Lower Specification limit",
                state_dict[dd_select]["lsl"],
                ud_lsl_input,
            ),
            build_value_setter_line(
                "value-setter-panel-ucl",
                "Upper Control limit",
                state_dict[dd_select]["ucl"],
                ud_ucl_input,
            ),
            build_value_setter_line(
                "value-setter-panel-lcl",
                "Lower Control limit",
                state_dict[dd_select]["lcl"],
                ud_lcl_input,
            ),
        ],
        state_value[dd_select]["usl"],
        state_value[dd_select]["lsl"],
        state_value[dd_select]["ucl"],
        state_value[dd_select]["lcl"],
    )


# ====== Callbacks to update stored data via click =====
@app.callback(
    output=Output("value-setter-store", "data"),
    inputs=[Input("value-setter-set-btn", "n_clicks")],
    state=[
        State("metric-select-dropdown", "value"),
        State("value-setter-store", "data"),
        State("ud_usl_input", "value"),
        State("ud_lsl_input", "value"),
        State("ud_ucl_input", "value"),
        State("ud_lcl_input", "value"),
    ],
)
def set_value_setter_store(set_btn, param, data, usl, lsl, ucl, lcl):
    if set_btn is None:
        return data
    else:
        data[param]["usl"] = usl
        data[param]["lsl"] = lsl
        data[param]["ucl"] = ucl
        data[param]["lcl"] = lcl

        # Recalculate ooc in case of param updates
        data[param]["ooc"] = populate_ooc(df[param], ucl, lcl)
        return data


@app.callback(
    output=Output("value-setter-view-output", "children"),
    inputs=[
        Input("value-setter-view-btn", "n_clicks"),
        Input("metric-select-dropdown", "value"),
        Input("value-setter-store", "data"),
        Input("interval-component", "n_intervals"),
    ],
)
def show_current_specs(n_clicks, dd_select, store_data,interval):
    if interval >= max_length:
        total_count = max_length - 1
    else:
        total_count = interval - 1
    store_data2 = df[0:total_count]
    x=store_data2[['pH','Temp','Voltage','Eduction','month','day','minutes','seconds']]
    y=store_data2['QC']
    analyse = {'DecisionTree':DecisionTree.predict(x),'LogisticRegression':LogisticRegression.predict(x),
               'RandomForest':RandomForest.predict(x),'XGboost':XGboost.predict(x)}
    if n_clicks > 0:
        curr_col_data = store_data[dd_select]
        new_df_dict = {
            "Specs": [
                "Upper Specification Limit",
                "Lower Specification Limit",
                "Upper Control Limit",
                "Lower Control Limit",
            ],
            "Models":[
                "DecisionTree",
                "LogisticRegression",
                "RandomForest",
                "XGboost",
            ],
            "Current Setup": [
                curr_col_data["usl"],
                curr_col_data["lsl"],
                curr_col_data["ucl"],
                curr_col_data["lcl"],
            ],
            "Current Accuracy":[
                round(accuracy_score(y,analyse['DecisionTree']),3),
                round(accuracy_score(y,analyse['LogisticRegression']),3),
                round(accuracy_score(y,analyse['RandomForest']),3),
                round(accuracy_score(y,analyse['XGboost']),3),
            ],
            "Current QC":
            [round(sum(DecisionTree.predict_proba(x)[:, 1])/total_count,3),
             round(sum(LogisticRegression.predict_proba(x)[:, 1])/total_count,3),
             round(sum(RandomForest.predict_proba(x)[:, 1])/total_count,3),
             round(sum(XGboost.predict_proba(x)[:, 1])/total_count,3),
            ],
        }
        new_df = pd.DataFrame.from_dict(new_df_dict)
        return dash_table.DataTable(
            style_header={"fontWeight": "bold", "color": "inherit"},
            style_as_list_view=True,
            fill_width=True,
            style_cell_conditional=[
                {"if": {"column_id": "Specs",}, "textAlign": "left"}
            ],
            style_cell={
                "backgroundColor": "#1e2130",
                "fontFamily": "Open Sans",
                "padding": "0 2rem",
                "color": "darkgray",
                "border": "none",
            },
            css=[
                {"selector": "tr:hover td", "rule": "color: #91dfd2 !important;"},
                {"selector": "td", "rule": "border: none !important;"},
                {
                    "selector": ".dash-cell.focused",
                    "rule": "background-color: #1e2130 !important;",
                },
                {"selector": "table", "rule": "--accent: #1e2130;"},
                {"selector": "tr", "rule": "background-color: transparent"},
            ],
            data=new_df.to_dict("rows"),
            columns=[{"id": c, "name": c} for c in ["Specs", "Current Setup","Models","Current Accuracy","Current QC"]],
        )


# decorator for list of output
def create_callback(param):
    def callback(interval, stored_data):
        count, ooc_n, ooc_g_value, indicator = update_count(
            interval, param, stored_data
        )
        spark_line_data = update_sparkline(interval, param)
        return count, spark_line_data, ooc_n, ooc_g_value, indicator

    return callback


for param in params[1:]:
    update_param_row_function = create_callback(param)
    app.callback(
        output=[
            Output(param + suffix_count, "children"),
            Output(param + suffix_sparkline_graph, "extendData"),
            Output(param + suffix_ooc_n, "children"),
            Output(param + suffix_ooc_g, "value"),
            Output(param + suffix_indicator, "color"),
        ],
        inputs=[Input("interval-component", "n_intervals")],
        state=[State("value-setter-store", "data")],
    )(update_param_row_function)


#  ======= button to choose/update figure based on click ============
@app.callback(
    output=Output("control-chart-live", "figure"),
    inputs=[
        Input("interval-component", "n_intervals"),
        Input(params[1] + suffix_button_id, "n_clicks"),
        Input(params[2] + suffix_button_id, "n_clicks"),
        Input(params[3] + suffix_button_id, "n_clicks"),
        Input(params[4] + suffix_button_id, "n_clicks"),
    ],
    state=[State("value-setter-store", "data"), State("control-chart-live", "figure")],
)
def update_control_chart(interval, n1, n2, n3, n4, data, cur_fig):
    # Find which one has been triggered
    ctx = dash.callback_context

    if not ctx.triggered:
        return generate_graph(interval, data, params[1])

    if ctx.triggered:
        # Get most recently triggered id and prop_type
        splitted = ctx.triggered[0]["prop_id"].split(".")
        prop_id = splitted[0]
        prop_type = splitted[1]

        if prop_type == "n_clicks":
            curr_id = cur_fig["data"][0]["name"]
            prop_id = prop_id[:-7]
            if curr_id == prop_id:
                return generate_graph(interval, data, curr_id)
            else:
                return generate_graph(interval, data, prop_id)

        if prop_type == "n_intervals" and cur_fig is not None:
            curr_id = cur_fig["data"][0]["name"]
            return generate_graph(interval, data, curr_id)


# Update piechart
@app.callback(
    output=Output("piechart", "figure"),
    inputs=[Input("interval-component", "n_intervals")],
    state=[State("value-setter-store", "data")],
)
def update_piechart(interval, stored_data):
    if interval == 0:
        return {
            "data": [],
            "layout": {
                "font": {"color": "white"},
                "paper_bgcolor": "rgba(0,0,0,0)",
                "plot_bgcolor": "rgba(0,0,0,0)",
            },
        }

    if interval >= max_length:
        total_count = max_length - 1
    else:
        total_count = interval - 1

    values = []
    colors = []
    for param in params[1:]:
        ooc_param = (stored_data[param]["ooc"][total_count] * 100)
        values.append(ooc_param)
        if ooc_param > 1:
            colors.append("#f45060")
        elif ooc_param > 3:
            colors.append("#ffcc00")

    new_figure = {
        "data": [
            {
                "labels": params[1:],
                "values": values,
                "type": "pie",
                "marker": {"colors": colors, "line": dict(color="white", width=2)},
                "hoverinfo": "label",
                "textinfo": "label",
            }
        ],
        "layout": {
            "margin": dict(t=20, b=50),
            "uirevision": True,
            "font": {"color": "white"},
            "showlegend": False,
            "paper_bgcolor": "rgba(0,0,0,0)",
            "plot_bgcolor": "resurgba(0,0,0,0)",
            "autosize": True,
        },
    }
    return new_figure

def generate_linechart():
    return dcc.Graph(
    id = "linechart",
    style={"width": "100%", "height": "120%"},
    config={
                    "staticPlot": False,
                    "editable": False,
                    "displayModeBar": False,
                },
    figure=go.Figure(
                    {
                        "data": [
                            {
                                "x": [],
                                "y": [],
                                "mode": "lines+markers",
                                "name": 'QC',
                                "line": {"color": "#f4d44d"},
                            }
                        ],
                        "layout": {
                            "uirevision": True,
                            "margin": dict(l=0, r=0, t=4, b=4, pad=0),
                            "xaxis": dict(
                                showline=False,
                                showgrid=False,
                                zeroline=False,
                                showticklabels=False,
                            ),
                            "yaxis": dict(
                                showline=False,
                                showgrid=False,
                                zeroline=False,
                                showticklabels=False,
                            ),
                            "paper_bgcolor": "rgba(0,0,0,0)",
                            "plot_bgcolor": "rgba(0,0,0,0)",
                        },
                    }
                ),
            )

@app.callback(
    output=Output("linechart", "figure"),
    inputs=[Input("interval-component", "n_intervals")],
    state=[State("value-setter-store", "data")],
)
def update_linecahrt(interval,stored_data):
    if interval >= max_length:
        total_count = max_length - 1
    else:
        total_count = interval - 1

    values = []
    x = df["Batch"][:total_count]
    QC_today =df["QC"][:total_count]
    ef = 0
    index = 0
    for i in QC_today:
        ef += i
        index +=1
        ef2 = ef/index
        values.append(ef2)
    new_figure = {
        "data": [
            {
                "x": x,
                "y": values,
                "mode": "lines+markers",
                "name": 'QC',
                "line": {"color": "#f4d44d"},
            }
        ],
        "layout": {
            "uirevision": True,
            "margin": dict(l=50, r=50, t=50, b=50, pad=10),
            "xaxis": dict(
                title='Batch number',
                titlefont=dict(
                size = 15,
                color = 'lightgray',),
                showgrid=True,
                autorange=True,
                zeroline=True,
                zerolinewidth=2,
                zerolinecolor='navy',
                gridcolor='#161a28',
            ),
            "yaxis": dict(
                title='efficiency',
                titlefont=dict(
                size = 15,
                color = 'lightgray'),
                showgrid=True,
                autorange=True,
                zeroline=True,
                zerolinewidth=2,
                zerolinecolor='navy',
                gridcolor='navy',
                            ),
            "paper_bgcolor": "rgba(0,0,0,0)",
            "plot_bgcolor": "rgba(0,0,0,0)",
        },
    }
    return new_figure

def generate_table():
    return html.Div(
            id="today_yesterday_total",
            children=[
                html.Div(
                    id="metric-select-menu",
                    # className='five columns',
                    children=[
                        html.Div(html.Table(id='value_setter_value_22',children=[ 
                            html.Tr([html.Td(['당일: '])]),
                            html.Br(),

                        ],style={"justifyContent": "center"})),
                        
                    ],
                )
            ],
        )

@app.callback(
    Output('today_yesterday_total', 'children'),
    inputs=[Input("interval-component", "n_intervals")],
    state=[State("value-setter-store", "data")])

def update_table(interval,stored_data):
    
    if interval >= max_length:
        total_count = max_length - 1
    else:
        total_count = interval - 1

    x = df["Batch"][:total_count]
    QC_today =df["QC"][:total_count]
    ef = 0
    index = 0
    for i in QC_today:
        ef += i
        index +=1
        ef2 = ef/index
    
    
    new_figure = html.Div(
            id="today_yesterday_total",
            children=[
                html.Div(
                    id="metric-select-menu",
                    # className='five columns',
                    children=[
                        html.Div(html.Table(id='value_setter_value_22',children=[ 
                            html.Tr([html.Td(['당일:',"%0.2f%%" % (round(ef2*100,2))])]),


                        ],style={"justifyContent": "center"})),
                        
                    ],
                )
            ],
        )
    return new_figure    

def generate_imagechart():
    return dcc.Graph(
                id="imagegraph_1",
                figure={
                    'data':[
                        {'x':[2],'y':[],
                        'type':'bar','name':'불량'},
                        {'x':[1],'y':[],
                        'type':'bar','name':'양품'}

                    ]})
def generate_imagechart2():
    return dcc.Graph(
                id="imagegraph_2",
                figure={'data':[{'x':[1],'y':[],'type':'bar','name':'불량1'},
                           {'x':[2],'y':[],'type':'bar','name':'불량2'},
                           {'x':[3],'y':[],'type':'bar','name':'불량3'},
]})


# Running the server
if __name__ == "__main__":
    app.run_server(debug=False, port=8050, use_reloader=False)